In [1]:
# Databricks notebook source

#Importando biblioteca
import pyspark.sql.functions as F

In [ ]:
#criando valores
dbutils.widgets.text("schema","")
dbutils.widgets.text("table","")

#Alterando valores inseridos
schema = dbutils.widgets.get("schema")
table = dbutils.widgets.get("table")

In [ ]:
#Consultando valores
df = spark.sql(f"select * from {schema}.{table}")
display(df)

In [ ]:
#Limpeza e transformação de dados
df_silver = (
    df.where("descricao is not null")
        .withColumn("row_number", F.expr("row_number() over(partition by codigo order by codigo)"))
        .where("row_number = 1")
        .drop("row_number")
        .withColumn("codigo", F.col("codigo").cast("string"))
        .withColumn("cpc", F.col("cpc").cast("string"))
        .withColumn("date_partition",F.current_date())
)

display(df_silver)

In [ ]:
#Salvando os dados
df_silver.write.mode("overwrite")\
         .format("delta")\
         .partitionBy("date_partition")\
         .option("compression","snappy")\
         .saveAsTable("tb_servicos_silver")